In [1]:
#import libraries used
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import fuzz
from bs4 import BeautifulSoup
import requests

In [2]:
#load the dataset
df = pd.read_csv('Drugs Master List.csv')
df.head(2)

,drug_name,medical_condition,side_effects,generic_name,drug_classes,brand_names,activity,rx_otc,pregnancy_category,csa,alcohol,related_drugs,medical_condition_description,rating,no_of_reviews,drug_link,medical_condition_url
0,doxycycline,Acne,"(hives, difficult breathing, swelling in your ...",doxycycline,"Miscellaneous antimalarials, Tetracyclines","Acticlate, Adoxa CK, Adoxa Pak, Adoxa TT, Alod...",87%,Rx,D,N,X,amoxicillin: https://www.drugs.com/amoxicillin...,Acne Other names: Acne Vulgaris; Blackheads; B...,6.8,760.0,https://www.drugs.com/doxycycline.html,https://www.drugs.com/condition/acne.html
1,spironolactone,Acne,hives ; difficulty breathing; swelling of your...,spironolactone,"Aldosterone receptor antagonists, Potassium-sp...","Aldactone, CaroSpir",82%,Rx,C,N,X,amlodipine: https://www.drugs.com/amlodipine.h...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.2,449.0,https://www.drugs.com/spironolactone.html,https://www.drugs.com/condition/acne.html


In [3]:
# replace value for salbutamol which contains html string
idx = df[df['drug_name'] == 'salbutamol'].index[0]
df.at[idx,'side_effects'] = np.nan

## Side Effects Cleaning

In [4]:
# function to remove similar string and duplicates
def remove_similar_duplicates(items):
    unique_items = []
    for item in items:
        is_duplicate = False
        for unique_item in unique_items:
            similarity_ratio = fuzz.ratio(item, unique_item)
            if similarity_ratio >= 40:  # Adjust the similarity threshold as per your requirements
                is_duplicate = True
                break
        if not is_duplicate:
            unique_items.append(item)
    return unique_items

In [5]:
# clean side effects with files containg "along" string pattern
def clean_along(col):
    txt = str(col)
    samp = txt.lower()
    samp = samp.split(":")[1:]

    sentt =" ".join(samp).split(".")[0]

    sentt = sentt.split("some side")[0]

    sentt =sentt.split("incidence not known")[:]

    if len(sentt[0]) > 1:
        sentt =sentt[:1]
    else:
        sentt = sentt[1]

    sentences = re.split(r"[;:]", "".join(sentt))

    filtered_items = [item.replace('rare', '').strip() for item in sentences]

    filtered_items = [item.replace('less common ', '').strip() for item in filtered_items]

    filtered_items = [item.replace('more common ', '').strip() for item in filtered_items]


    filtered_items = [item.strip() for item in filtered_items if item.strip()]

    filtered_items = filtered_items = [item[3:].strip() if item.startswith('or ') else item for item in filtered_items]

    return filtered_items

In [6]:
# words to exclude in a search string
exclude_words = ['have','occur','include','known','provider','healthcare',
                 'may','sometime','stop','form ', 'children','1088',
                 'any','following','month','days',
                 'include','reaction','last dose',
                 'if you', 'allergic reaction','warning','notice','along','side effects', 'doctor', 'signs of','questions']

# clean side effects string based on split values
def clean_side_effect(col_value,split = "have"):
    
    splited_text = col_value.split(split)[1:]

    sentences = re.split(r"[.;:)(]", " ".join(splited_text))

    filtered_items = [item for item in sentences if not any(word in item for word in exclude_words)]

    filtered_items = [item.strip() for item in filtered_items if item.strip()]

    filtered_items =[item.split(",") for item in filtered_items]
    # flatten list
    filtered_items = [element for sublist in filtered_items for element in sublist]

    # replace  values
    filtered_items = [item.replace('severe', '').strip() for item in filtered_items]
    filtered_items = [item.replace('more common', '').strip() for item in filtered_items]
    filtered_items = [item.replace('especially on one side of the body', '').strip() for item in filtered_items]

    # remove or from words start
    filtered_items = filtered_items = [item[3:].strip() if item.startswith('or ') else item for item in filtered_items]
    # remove duplicate/similar words to reduce redudancy
    filtered_items = remove_similar_duplicates(filtered_items)
    # remove words less than length 2
    filtered_items = [item.strip() for item in filtered_items if len(item)>2]

    return filtered_items

In [7]:
#another function with split and clean strategy
def clean_simple_text(col_values):
    
    filtered_items = re.split(r"[.;,:)(]", col_values)
    
    filtered_items = [item.strip() for item in filtered_items if item.strip()]
    filtered_items = filtered_items = [item[3:].strip() if item.startswith('or ') else item for item in filtered_items]
    # remove duplicate/similar words to reduce redudancy
    filtered_items = remove_similar_duplicates(filtered_items)
    
    filtered_items = [item.strip() for item in filtered_items if len(item)>2]

    return filtered_items

In [8]:
#function to clean the side effect dataframe col, trying out multiple split and clean strategy
def clean_sfx_column(col_value):
    
    if col_value is np.nan:
        return []

    col_value = str(col_value).strip().lower()

    if col_value[:5] == "along":
        return clean_along(col_value)

    filtered_items= clean_side_effect(col_value,split="have")

    if len(filtered_items) < 3:
        filtered_items = clean_side_effect(col_value,split="occur")

    if len(filtered_items) < 2:
        filtered_items = clean_side_effect(col_value,split="known")

    if len(filtered_items) < 3:
        filtered_items = clean_side_effect(col_value,split="include")
    
    if len(filtered_items) < 1:
        filtered_items = clean_simple_text(col_value)

    return filtered_items

In [9]:
# get data info 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   drug_name                      999 non-null    object 
 1   medical_condition              999 non-null    object 
 2   side_effects                   970 non-null    object 
 3   generic_name                   987 non-null    object 
 4   drug_classes                   978 non-null    object 
 5   brand_names                    613 non-null    object 
 6   activity                       999 non-null    object 
 7   rx_otc                         999 non-null    object 
 8   pregnancy_category             927 non-null    object 
 9   csa                            999 non-null    object 
 10  alcohol                        504 non-null    object 
 11  related_drugs                  491 non-null    object 
 12  medical_condition_description  999 non-null    obj

In [10]:
%%time
df['side_effects'] = df['side_effects'].apply(clean_sfx_column) # apply clean side effects to side_effects column

CPU times: user 407 ms, sys: 6.47 ms, total: 413 ms
Wall time: 496 ms


In [11]:
df['side_effects']

0      [skin rash, fever, flu-like symptoms, muscle a...
1      [a light-headed feeling, like you might pass o...
2      [little or no urination, swelling in your feet...
3      [nosebleeds, headache, back pain, cold symptom...
4      [redness, itching, dryness of treated skin are...
                             ...                        
994    [chest pain, fast, slow, uneven heart rate, di...
995    [hives, difficult breathing, swelling of your ...
996    [hives, itching, red, swollen, peeling skin wi...
997    [noisy breathing, drowsiness, hallucinations, ...
998    [dizziness, trouble sleeping, high blood press...
Name: side_effects, Length: 999, dtype: object

In [12]:
df['sfx_count'] = df['side_effects'].apply(lambda a: len(a))

## Missing side effects

In [13]:
# stratedy to find side effect from html
def find_elements(soup, common):
    common_text = common#"More common"

    # Find all elements containing the common text
    elements = soup.find_all(string=common_text)

    # Find the first list after the common text
    target_list = None
    for element in elements:
        if element.find_next("ul"):
            target_list = element.find_next("ul")
            break
    
    effects = []
    # Print the contents of the first list
    if target_list:
        for li in target_list.find_all("li"):
            effects.append(li.text.strip())
    else:
        pass
    
    return effects

In [14]:
# stratedy to find side effect from html
def sfx_from_paragraph(soup, common_text = "Side effects include:", find = "p"):


    # Common text to search for
    common_text = common_text

    # Find all <b> tags containing the common text
    found_p = soup.find_all(find, string=common_text)

    # Retrieve the values within the paragraphs containing the <b> tags
    values = []
    for p in found_p:
        paragraph = p.find_next("p")
        if paragraph:
            value = paragraph.text.replace(common_text, "").strip()
            values.extend(value.split(","))

    return values

In [15]:
# words to exclude in resulting array
exclude_words = ['have','occur','include','known','provider','healthcare',
                 'may','sometime','stop','form ', 'children','1088',
                 'any','following','month','days',
                 'include','reaction','last dose','Ref','manufacturer',
                 'if you', 'allergic reaction','warning','notice','along','side effects', 'doctor', 'signs of','questions']

#clean the scraped side effects text 
def clean_scraped_sfx(col_value):
    
    text = "#".join(col_value)
    #remove \ns
    text = re.sub(r'\n', '', text)

    # Remove multiple whitespaces
    text = re.sub(r'\s+', ' ', text)

    sentences = re.split(r"[.;:)(#]",text)

    filtered_items = [item for item in sentences if not any(word in item for word in exclude_words)]

    filtered_items = [item.strip() for item in filtered_items if item.strip()]

    #filtered_items =[item.split(",") for item in filtered_items]
    # flatten list
    #filtered_items = [element for sublist in filtered_items for element in sublist]

    # replace  values
    filtered_items = [item.replace('severe', '').strip() for item in filtered_items]
    filtered_items = [item.replace('more common', '').strip() for item in filtered_items]
    filtered_items = [item.replace('especially on one side of the body', '').strip() for item in filtered_items]

    # remove or from words start
    filtered_items = filtered_items = [item[3:].strip() if item.startswith('or ') else item for item in filtered_items]
    # remove duplicate/similar words to reduce redudancy
   # filtered_items = remove_similar_duplicates(filtered_items)
    # remove words less than length 2
    filtered_items = [item.strip() for item in filtered_items if len(item)>2]

    return filtered_items

In [16]:
# scrape side effects from url
def scrape_sfx(url):
    # Send a GET request to the web page
    response = requests.get(url)
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'lxml')
    
    sfx_link = None
    for link in soup.find_all('a'):
        try:
            newlink = link.get('href')
            #print(newlink)
        except:
             newlink = None
        if newlink is not None and 'side-effects' in newlink :
            sfx_link = link.get('href')
            #print(newlink)
            #print(sfx_link)
            break
    
    if sfx_link:
        sfx_url = "https://www.drugs.com" + sfx_link if not sfx_link.startswith('http') else sfx_link
        sfx_response = None
        try:
            #print(sfx_url)
            sfx_response = requests.get(sfx_url)
            if sfx_response.status_code != 200:
                return []
        except:
            #print(sfx_url)
            return []
            # sfx_url = url + sfx_link 
            # print(sfx_url)
            # sfx_response = requests.get(sfx_url)
    else:
        return []

        
    
    soup = BeautifulSoup(sfx_response.text, 'lxml')
    
    effects = find_elements(soup,"Serious side effects")
    
    effects2 = []
    if len(effects) < 2:
        effects2 = find_elements(soup,"Less common")
    
    if len(effects2) > len(effects):
        effects = effects2

    
    if len(effects) < 2:
        effects2 = find_elements(soup,"More common")
    
    if len(effects2) > len(effects):
        effects = effects2
    
    if len(effects) < 1:
        effects = sfx_from_paragraph(soup)
    
    if len(effects) < 1:
        effects = sfx_from_paragraph(soup,common_text = "General", find = "h3")
        
    return effects

### Cleaning missing side effects
The missing links are extracted from the dataset and cleaned one by one.
Some links with had different patterns where resulted to blank arrays.

In [17]:
# get missing drug links and index
idx_link_ = df[df['sfx_count']==0]['drug_link'].index.to_list()

links_ = df[df['sfx_count']==0]['drug_link'].to_list()

In [18]:
%time
## fill missing side effects
for i, url in enumerate(links_):
    txt = clean_scraped_sfx(scrape_sfx(url))
    df.at[idx_link_[i],'side_effects'] = txt
    print(txt)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.72 µs
['Black, tarry stools', 'blistering, peeling, or loosening of the skin', 'changes in skin color', 'chest pain or tightness', 'chills', 'clay-colored stools', 'cough or hoarseness', 'dark urine', 'diarrhea', 'dizziness', 'fever', 'general feeling of tiredness or weakness', 'headache', 'itching, skin rash', 'joint or muscle pain', 'light-colored stools', 'loss of appetite', 'lower back or side pain', 'nausea', 'pain, tenderness, or swelling of the foot or leg', 'painful or difficult urination', 'pale skin', 'red skin lesions, often with a purple center', 'red, irritated eyes', 'sore throat', 'sores, ulcers, or white spots in the mouth or on the lips', 'stomach pain', 'swollen or painful glands', 'trouble breathing', 'unpleasant breath odor', 'unusual bleeding or bruising', 'vomiting of blood', 'yellow eyes or skin']
['Peeling', 'redness']
['Peeling', 'redness']
['GI effects', 'anorexia', 'epigastric distress', 'nausea', 'vom

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   drug_name                      999 non-null    object 
 1   medical_condition              999 non-null    object 
 2   side_effects                   999 non-null    object 
 3   generic_name                   987 non-null    object 
 4   drug_classes                   978 non-null    object 
 5   brand_names                    613 non-null    object 
 6   activity                       999 non-null    object 
 7   rx_otc                         999 non-null    object 
 8   pregnancy_category             927 non-null    object 
 9   csa                            999 non-null    object 
 10  alcohol                        504 non-null    object 
 11  related_drugs                  491 non-null    object 
 12  medical_condition_description  999 non-null    obj

**drop irreleavant columns**

In [20]:
df.drop(['medical_condition_url','drug_link','no_of_reviews','sfx_count'],axis=1, inplace=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   drug_name                      999 non-null    object 
 1   medical_condition              999 non-null    object 
 2   side_effects                   999 non-null    object 
 3   generic_name                   987 non-null    object 
 4   drug_classes                   978 non-null    object 
 5   brand_names                    613 non-null    object 
 6   activity                       999 non-null    object 
 7   rx_otc                         999 non-null    object 
 8   pregnancy_category             927 non-null    object 
 9   csa                            999 non-null    object 
 10  alcohol                        504 non-null    object 
 11  related_drugs                  491 non-null    object 
 12  medical_condition_description  999 non-null    obj

## Handling other missing columns
<b>Note:</b> Similar approach taken to handle missing side effects can be used, to scrape 
the missing drug_classes, brand_name, related_drugs and ratings.

But due to my limited availabilty to work on this "Case Study Assessment" i will be ignoring these missing columns.

In [22]:
df['pregnancy_category'].value_counts()

pregnancy_category
C    444
N    168
B    159
D     95
X     61
Name: count, dtype: int64

## Clean Other Columns

In [23]:
# cleans the class brand column
def clean_class_brand(col_value):
    
    if col_value is np.nan:
        return []

    item_lst = []
    item_lst = re.split(r"[.;:),/(#]", str(col_value))
    item_lst = [item.strip() for item in item_lst if item.strip()]

    return item_lst

In [24]:
df['drug_classes'] = df['drug_classes'].apply(clean_class_brand)
df['drug_classes']

0           [Miscellaneous antimalarials, Tetracyclines]
1      [Aldosterone receptor antagonists, Potassium-s...
2                                        [Tetracyclines]
3      [Miscellaneous antineoplastics, Miscellaneous ...
4         [Topical acne agents, Vaginal anti-infectives]
                             ...                        
994                     [Upper respiratory combinations]
995                                [Topical rubefacient]
996                     [Upper respiratory combinations]
997                     [Upper respiratory combinations]
998                     [Upper respiratory combinations]
Name: drug_classes, Length: 999, dtype: object

In [25]:
df['brand_names'] = df['brand_names'].apply(clean_class_brand)
df['brand_names']

0      [Acticlate, Adoxa CK, Adoxa Pak, Adoxa TT, Alo...
1                                  [Aldactone, CaroSpir]
2      [Dynacin, Minocin, Minolira, Solodyn, Ximino, ...
3                                                     []
4      [Cleocin T, Clindacin ETZ, Clindacin P, Clinda...
                             ...                        
994    [Mapap Cold Formula, Robitussin Peak Cold Dayt...
995    [Anti-Itch Steroid Free, Arctic Relief, Avedan...
996                                                   []
997    [Ambifed CD, Biotussin DAC, Cheratussin DAC, L...
998             [Aphedrid, A-Phedrin, Aprodine, Vi-Sudo]
Name: brand_names, Length: 999, dtype: object

In [26]:
#cleans related drugs col
def clean_related_drug(col_value):
    
    if col_value is np.nan:
        return []
    
    
    item_lst = []
    item_lst = re.split(r"[|]",str(col_value))
    item_lst = [item.split(":")[0].strip() for item in item_lst]
    
    return item_lst

In [27]:
df['related_drugs'] = df['related_drugs'].apply(clean_related_drug)
df['related_drugs']

0      [amoxicillin, prednisone, albuterol, ciproflox...
1      [amlodipine, lisinopril, losartan, metoprolol,...
2      [amoxicillin, prednisone, doxycycline, ciprofl...
3      [doxycycline, clindamycin topical, erythromyci...
4      [doxycycline, metronidazole, metronidazole top...
                             ...                        
994                                                   []
995    [Buprenex, aspirin, acetaminophen, tramadol, T...
996    [Benadryl, diphenhydramine, chlorpheniramine, ...
997    [Benadryl, diphenhydramine, chlorpheniramine, ...
998                                                   []
Name: related_drugs, Length: 999, dtype: object

fill the missing column with "N/A" and 0 for float values (ratings)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   drug_name                      999 non-null    object 
 1   medical_condition              999 non-null    object 
 2   side_effects                   999 non-null    object 
 3   generic_name                   987 non-null    object 
 4   drug_classes                   999 non-null    object 
 5   brand_names                    999 non-null    object 
 6   activity                       999 non-null    object 
 7   rx_otc                         999 non-null    object 
 8   pregnancy_category             927 non-null    object 
 9   csa                            999 non-null    object 
 10  alcohol                        504 non-null    object 
 11  related_drugs                  999 non-null    object 
 12  medical_condition_description  999 non-null    obj

## Missing values handling

In [30]:
# fill the missing values with "N/A" for Not avaialbe for string and 0 for integer
df[['generic_name','pregnancy_category','alcohol']] = df[['generic_name','pregnancy_category','alcohol']].fillna("N/A")
df['rating'] = df['rating'].fillna(0)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   drug_name                      999 non-null    object 
 1   medical_condition              999 non-null    object 
 2   side_effects                   999 non-null    object 
 3   generic_name                   999 non-null    object 
 4   drug_classes                   999 non-null    object 
 5   brand_names                    999 non-null    object 
 6   activity                       999 non-null    object 
 7   rx_otc                         999 non-null    object 
 8   pregnancy_category             999 non-null    object 
 9   csa                            999 non-null    object 
 10  alcohol                        999 non-null    object 
 11  related_drugs                  999 non-null    object 
 12  medical_condition_description  999 non-null    obj

In [33]:
df.drug_name = df.drug_name.str.lower().str.strip()

In [34]:
# save data to fastparquet
df.to_parquet('clean_drug_master.parquet', engine='fastparquet', index=False)